In [3]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from src.pipeline.models import (
    DataSourceConfig,
    ProcessingStepConfig,
    OutputConfig,
    PipelineConfig
)
from pydantic import ValidationError
import yaml

ModuleNotFoundError: No module named 'src'

In [ ]:
import yaml

with open('pipeline_config.yaml', 'r') as f:
    config = yaml.safe_load(f)

processing_steps = []
for step_config in config['processing_steps']:
    step = ProcessingStep(
        name=step_config['name'],
        function=globals()[step_config['function']],
        inputs=step_config['inputs'],
        output_table=step_config['output_table']
    )
    processing_steps.append(step)

pipeline = Pipeline(processing_steps)


In [ ]:
# Widget for Data Source Name
ds_name = widgets.Text(description='Name')

# Widget for Data Source Type
ds_type = widgets.Dropdown(
    options=['volume', 'database', 'api'],
    description='Type'
)

# Widget for Path
ds_path = widgets.Text(description='Path')

# Widgets for Format, Catalog, Schema, Table
ds_format = widgets.Text(description='Format')
ds_catalog = widgets.Text(description='Catalog')
ds_schema = widgets.Text(description='Schema')
ds_table = widgets.Text(description='Table')

# Button to Add Data Source
add_ds_button = widgets.Button(description='Add Data Source')

# Output area for Data Source
ds_output = widgets.Output()

In [ ]:
data_sources = []

def on_add_ds_clicked(b):
    with ds_output:
        clear_output()
        try:
            ds_config = DataSourceConfig(
                name=ds_name.value,
                type=ds_type.value,
                path=ds_path.value,
                format=ds_format.value,
                catalog=ds_catalog.value,
                schema=ds_schema.value,
                table=ds_table.value
            )
            data_sources.append(ds_config)
            print(f"Added Data Source: {ds_config.name}")
        except ValidationError as e:
            print("Validation Error:")
            print(e)


In [ ]:
add_ds_button.on_click(on_add_ds_clicked)

In [ ]:
ds_widget_box = widgets.VBox([
    ds_name,
    ds_type,
    ds_path,
    ds_format,
    ds_catalog,
    ds_schema,
    ds_table,
    add_ds_button,
    ds_output
])

display(ds_widget_box)


In [ ]:
assemble_button = widgets.Button(description='Assemble Pipeline')
assemble_output = widgets.Output()

def on_assemble_clicked(b):
    with assemble_output:
        clear_output()
        try:
            pipeline_config = PipelineConfig(
                data_sources=data_sources,
                processing_steps=processing_steps,
                outputs=outputs
            )
            print("Pipeline Configuration Assembled Successfully!")
            # Optionally display or save the configuration
            config_dict = pipeline_config.dict()
            print(yaml.dump(config_dict))
        except ValidationError as e:
            print("Validation Error in Pipeline Configuration:")
            print(e)

assemble_button.on_click(on_assemble_clicked)

display(assemble_button, assemble_output)


In [ ]:
run_pipeline_button = widgets.Button(description='Run Pipeline')

def on_run_pipeline_clicked(b):
    with assemble_output:
        clear_output()
        try:
            pipeline_config = PipelineConfig(
                data_sources=data_sources,
                processing_steps=processing_steps,
                outputs=outputs
            )
            pipeline = Pipeline(pipeline_config)
            pipeline.execute()
            print("Pipeline executed successfully.")
        except Exception as e:
            print("Error executing pipeline:")
            print(e)

run_pipeline_button.on_click(on_run_pipeline_clicked)

display(run_pipeline_button)


In [ ]:
save_button = widgets.Button(description='Save Configuration')

def on_save_clicked(b):
    with assemble_output:
        clear_output()
        try:
            pipeline_config = PipelineConfig(
                data_sources=data_sources,
                processing_steps=processing_steps,
                outputs=outputs
            )
            config_dict = pipeline_config.dict()
            with open('config/generated_pipeline_config.yaml', 'w') as f:
                yaml.dump(config_dict, f)
            print("Configuration saved to 'config/generated_pipeline_config.yaml'.")
        except ValidationError as e:
            print("Validation Error:")
            print(e)

save_button.on_click(on_save_clicked)

display(save_button)


In [ ]:
load_button = widgets.Button(description='Load Configuration')
config_file_input = widgets.Text(description='Config File', value='config/pipeline_config.yaml')

def on_load_clicked(b):
    with assemble_output:
        clear_output()
        try:
            with open(config_file_input.value, 'r') as f:
                config_data = yaml.safe_load(f)
            # Clear existing data
            data_sources.clear()
            processing_steps.clear()
            outputs.clear()
            # Populate data sources
            for ds in config_data.get('data_sources', []):
                ds_config = DataSourceConfig(**ds)
                data_sources.append(ds_config)
                # Update widgets accordingly (if desired)
            # Similarly for processing steps and outputs
            print("Configuration loaded successfully.")
        except Exception as e:
            print("Error loading configuration:")
            print(e)

load_button.on_click(on_load_clicked)

display(config_file_input, load_button)
